In [1]:
import os
import glob
import pandas as pd

In [12]:
old = pd.read_csv("../../src/processing_area_june_23.csv")
new = pd.read_csv("../../src/processing_area_june_28.csv")

In [13]:
import numpy as np
len(np.unique(new['Y']))

1644

In [22]:
new['Y'] = np.around(new['Y'], 12)
old['Y'] = np.around(old['Y'], 12)
new['X'] = np.around(new['X'], 12)
old['X'] = np.around(old['X'], 12)

In [27]:
old_y = np.unique(old['Y'])
new_y = np.unique(new['Y'])

old_x = np.unique(old['X'])
new_x = np.unique(new['X'])

to_remove_y = [x for x in new_y if x not in old_y]
to_remove_x = [x for x in new_x if x not in old_x]

for i in to_remove_y:
    new = new[new.Y != i]
    
for i in to_remove_x:
    new = new[new.X != i]

In [28]:
new.to_csv("temp.csv")

In [29]:
new = pd.merge(new, old, on = ['X', 'Y'], how = 'left')

In [30]:
def id_tile_number(new, old, idx):
    if np.isnan(new['Y_tile'][idx]):
        new_y = new['Y'][idx]
        new_x = new['X'][idx]
        y_tile = np.unique(old[old['Y'] == new_y]['Y_tile'])
        x_tile = np.unique(old[old['X'] == new_x]['X_tile'])
        assert len(y_tile) == 1
        assert len(x_tile) == 1
        new['X_tile'][idx] = x_tile[0]
        new['Y_tile'][idx] = y_tile[0]


In [31]:
from tqdm.notebook import tnrange
for row in tnrange(len(new)):
    id_tile_number(new, old, row)

/Users/john.brandt/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/john.brandt/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [33]:
new.to_csv("processing_area_june_28.csv")

In [71]:
base_folder = "../project-monitoring/tof/"
from shutil import copyfile, copytree

for folder_y in os.listdir(base_folder):
    if "." not in folder_y:
        for folder_x in os.listdir(base_folder + folder_y):
            if "." not in folder_x:
                for file in os.listdir(base_folder + folder_y + "/" + folder_x):
                    if file[-4:] == ".tif":
                        source_file = file
                        source = old[old['X_tile'].astype(str) == folder_y]
                        source = source[source['Y_tile'].astype(str) == folder_x]
                        source = source.reset_index(drop = True)
                        source_y = source['Y'][0]
                        source_x = source['X'][0]
                        dest = new[np.isclose(new['Y'], source_y)]
                        dest = dest[np.isclose(dest['X'], source_x)]
                        dest = dest.reset_index(drop = True)
                        dest_x = dest['X_tile'][0]
                        dest_y = dest['Y_tile'][0]
                        #print(folder_y, dest_x)
                        #print(folder_x, dest_y)
                        
                        new_dir = "../project-monitoring/tiles/" + str(dest_x) + "/" + str(dest_y) + "/"
                        new_file = f'{new_dir}{dest_x}X{dest_y}Y_v1.tif'
                        old_file = f'{base_folder}{folder_y}/{folder_x}/{file}'
                        
                        
                        if not os.path.exists(os.path.realpath(new_dir)):
                            os.makedirs(os.path.realpath(new_dir))
                        copyfile(old_file, new_file)

# Create temp copy of individual country for reprocessing

In [4]:
df = new[new['country'] == 'Rwanda']
df.head(5)

,Y,X,country,Y_tile,X_tile
138843,-1.138889,30.416667,Rwanda,943,2233
138844,-1.194444,30.305556,Rwanda,942,2231
138845,-1.194444,30.361111,Rwanda,942,2232
138846,-1.194444,30.416667,Rwanda,942,2233
138847,-1.250000,30.250000,Rwanda,941,2230


In [10]:
from shutil import copyfile, copytree

for index, row in df.iterrows():
    x = str(int(row['X_tile']))
    y = str(int(row['Y_tile']))
    old_path = f'../../project-monitoring/tof-output/{str(x)}/{str(y)}/'
    new_path = f'../../project-monitoring/rwanda/{str(x)}/{str(y)}/'
    if os.path.isdir(old_path):
        copytree(old_path, new_path)
